In [ ]:
# !pip install lyricsgenius

### 데이터 수집

In [1]:
import lyricsgenius as lg
import re
import pandas as pd
import json
import numpy as np

In [ ]:
TOKEN = "#####################################"

genius = lg.Genius(TOKEN, timeout=20, sleep_time=10) 

# timeout 문제 해결: timeout, sleep_time 늘리기
# https://github.com/johnwmillr/LyricsGenius/issues/121#issuecomment-704448192

end_page = 5
song_dataset = {"artists":[], "featured_artists":[], "title":[], "genre":[], "lyrics":[], 'url':[]}

tags = ['rap', 'pop', 'r-b', 'rock', 'country', 'k-hip-hop', 'k-r-b', 'k-rock', 'k-ballad',
        'k-pop', 'acoustic', 'comedy', 'dance', 'disco', 'disney', 'jazz', 'soul']

for tag_name in tags:
    print("<",tag_name,">")

    for page in range(1, end_page+1):
        res = genius.tag(tag_name, page=page) # 장르와 page에 맞춰 데이터 수집 - 1페이지 당 최대 20개 곡
        i = (20*(page-1)) + 1

        for hit in res['hits']: # 한 곡씩
            song_dataset['artists'].append(hit['artists']) # 가수
            song_dataset['title'].append(hit['title']) # 제목
            song_dataset['featured_artists'].append(hit['featured_artists']) # 피처링

            if tag_name in ['k-hip-hop', 'k-r-b', 'k-rock', 'k-ballad']: # 한국 노래 장르 이름 수정(k- 제거)
                tag_name = tag_name[2:]
            song_dataset['genre'].append(tag_name) # 장르

            song_dataset['url'].append(hit['url']) # genius url

            song_lyrics = genius.lyrics(song_url=hit['url']) # url에서 가사 수집

            song_dataset['lyrics'].append(song_lyrics)

            print(i, end = " ")
            i+=1
        #page = res['next_page']
    print()

### 데이터 전처리

In [15]:
song_df = pd.DataFrame(song_dataset)
song_df.head()

,artists,featured_artists,title,genre,lyrics,url
0,[Eminem],[],Rap God,rap,"Rap God Lyrics[Intro]\n""Look, I was gonna go e...",https://genius.com/Eminem-rap-god-lyrics
1,[Cardi B],[Megan Thee Stallion],WAP,rap,"WAP Lyrics[Intro: Cardi B, Al ""T"" McLaran & Me...",https://genius.com/Cardi-b-wap-lyrics
2,[Kendrick Lamar],[],HUMBLE.,rap,HUMBLE. Lyrics[Intro]\nNobody pray for me\nIt ...,https://genius.com/Kendrick-lamar-humble-lyrics
3,[Migos],[Lil Uzi Vert],Bad and Boujee,rap,Bad and Boujee Lyrics[Intro: Offset & Future]\...,https://genius.com/Migos-bad-and-boujee-lyrics
4,[Drake],[],God's Plan,rap,God’s Plan Lyrics[Intro]\nAnd they wishin' and...,https://genius.com/Drake-gods-plan-lyrics


In [ ]:
# 전처리 전 데이터 저장: csv, json 형태
song_df.to_csv("data/bf_preprocessing.csv", index=False)

with open("data/bf_preprocessing.json", 'w') as file:
    json.dump(song_dataset, file)

In [24]:
song_df.head()

,artists,featured_artists,title,genre,lyrics,url
0,[Eminem],[],Rap God,rap,"Rap God Lyrics[Intro]\n""Look, I was gonna go e...",https://genius.com/Eminem-rap-god-lyrics
1,[Cardi B],[Megan Thee Stallion],WAP,rap,"WAP Lyrics[Intro: Cardi B, Al ""T"" McLaran & Me...",https://genius.com/Cardi-b-wap-lyrics
2,[Kendrick Lamar],[],HUMBLE.,rap,HUMBLE. Lyrics[Intro]\nNobody pray for me\nIt ...,https://genius.com/Kendrick-lamar-humble-lyrics
3,[Migos],[Lil Uzi Vert],Bad and Boujee,rap,Bad and Boujee Lyrics[Intro: Offset & Future]\...,https://genius.com/Migos-bad-and-boujee-lyrics
4,[Drake],[],God's Plan,rap,God’s Plan Lyrics[Intro]\nAnd they wishin' and...,https://genius.com/Drake-gods-plan-lyrics


In [ ]:
# 리스트 형식의 artists 데이터 변환
song_df['artists'] = song_df['artists'].apply(lambda x: ','.join(x) if len(x) else x)

In [ ]:
# 리스트 형식의  featured_artists 데이터 변환
song_df['featured_artists'] = song_df['featured_artists'].apply(lambda x: np.nan if x == [] else x)
song_df['featured_artists'] = song_df['featured_artists'].apply(lambda x: ','.join(x) if (type(x) == list and len(x)) else x)

In [ ]:
## 노이즈 제거
song_df['new_lyrics'] = song_df['lyrics'].apply(lambda x: re.sub(r'\[.*\]*', "", x)) # 노래 흐름([코러스1], [벌스1], ...) 표기 제거

song_df['new_lyrics'] = song_df['new_lyrics'].apply(lambda x: re.sub("\u205f", " ", x))
song_df['new_lyrics'] = song_df['new_lyrics'].apply(lambda x: re.sub(r'[0-9]\.[0-9][a-zA-z]+Embed$', "", x)) # 가사 끝에 가사와 관련 없는 부분 제거 - 경우1
song_df['new_lyrics'] = song_df['new_lyrics'].apply(lambda x: re.sub(r'[0-9]+Embed$', "", x)) # 가사 끝에 가사와 관련 없는 부분 제거 - 경우2

# 제목+가사의 시작임을 알리는 가사 외 부분 제거
song_df['new_lyrics'] = song_df[['title', 'new_lyrics']].apply(lambda x: re.sub(f"^{x['title']} Lyrics\n", "", x['new_lyrics']), axis=1)

song_df['new_lyrics'] = song_df['new_lyrics'].apply(lambda x: re.sub('\n', ". ", x)) # 줄 바꿈은 space로 변경

In [ ]:
song_df.to_csv("data/music_datasets.csv", index=False)